In [222]:
# !pip3 install openai
#!pip install python-dotenv
#!pip install langchain
#!pip install langchain-community
#!pip install neo4j
#!pip install json
#!pip install --upgrade langchain
#!pip install tiktoken
#!pip install py2neo

In [1]:
from dotenv import load_dotenv
import os
import json
from neo4j import GraphDatabase
from langchain.chains import GraphQAChain
from langchain.chat_models import ChatOpenAI
from langchain_community.graphs import Neo4jGraph
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.vectorstores import Neo4jVector
from langchain.embeddings.openai import OpenAIEmbeddings
from py2neo import Graph

# Warning control
import warnings
warnings.filterwarnings("ignore")

In [2]:
NEO4J_URL = ""
NEO4J_USERNAME = ""
NEO4J_PASSWORD = ""
NEO4J_DATABASE = ""

In [3]:
driver = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [4]:
kg = Neo4jGraph(
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database=NEO4J_DATABASE
)

In [2]:
import json

from openai import OpenAI

## Put the key here.
openai_key = ""
openai_client = OpenAI(api_key=openai_key)

def call_gpt4o_mini(message):
    """Call the GPT model for text information and return the response."""
    try:
        response = openai_client.chat.completions.create(
            model = "gpt-4o-mini",
            messages=[{"role": "user", 
                       "content": message}],
            temperature=0.0,
            max_tokens=4000
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error calling GPT: {e}")
        return None

In [11]:
# Test the GPT model
call_gpt4o_mini("What is the capital of France?")

'The capital of France is Paris.'

In [18]:
prompt = {
    "Task": "You are an information extraction tool. Your task is to analyze the given Reddit post and extract meaningful entities, properties, and relationships to create a knowledge graph about PCSK9 inhibitors in Neo4j. Here is the posting: {posting}",
    "Instruction": """
    Return a JSON object with the following structure:    
    data = {
    "entities": [
        {
            "label": "User",
            "properties": {
                "name": "",
                "score": ,
                "date": ""
            }
        },
        {
            "label": "",
            "properties": {
                "name": "",
                "description": ""
            }
        }
    ],
    "relationships": [
        {
            "start": {
                "label": "User",
                "properties": {"name": ""}
            },
            "end": {
                "label": "",
                "properties": {"name": ""}
            },
            "type": ""
        },
        ]
    }

    Ensure the following:
    - Focus on extracting entities related to PCSK9 inhibitors, such as health effects, insurance, financial burdens, or other life aspects.
    - Avoid specific numbers (e.g., costs, dosages, cholesterol levels) in entity names; include them in properties instead.
    - All relationships must clearly define the logical or functional dependencies between the entities and PCSK9 inhibitors.
    - If you cannot identify any entity, return nothing. It is possible that the text contains no meaningful information.
    - Ensure that all extracted entities and relationships are factual and verifiable directly from the text itself.
    - Maintain language style consistency in the terminology used to describe these entities and relationships.
    - Please extract only meaningful entities and relationships from the text.
    - Do not use any external knowledge or assumptions.
    - Please follow the language style of the given below examples strictly.
    - Return only the JSON object without any additional explanation.

    - Nodes can only have the following labels: 'User', 'Medication', 'Condition', Insurance', and 'Attribute'.
    - Each node must adhere to the following property rules:
        - ** 'User' nodes **: Must contain exactly these properties: 'name', 'score', and 'date'.
        - ** Other nodes **: Must only contain two properties: 'name' and 'description'.
        
    - Relationships can only have the following types:
        - ** 'SUFFERS_FROM' **: Used to describe a user's negative health-related condition.
        - ** 'EXPERIENCED' **: Used to describe a positive or neutral event or experience a user has gone through (e.g., decrease in cholesterol levels experienced, improvement in mood experienced, insurance issues experienced).
        - ** 'MENTIONS' **: Used to describe general attributes or additional details mentioned by the user (e.g., insurance, cost, administration methods).
        - ** 'USED' **: Used to describe a user's medication.
    - All relationships must directly connect from the 'User' node.

    Example 1:
    
    Input:
    {
    "user_id": "chrisenvy",
    "score": 1,
    "date": "2024-04-04",
    "text": "I take Praluent and have taken Repatha in the past (switched insurances and one would only cover one and the other insurance would only cover the other). That being said, I am allergic to statins and when I take them, it damages my liver. I've had 3 different doctors try to put me on statins (they never learn, lol). Anyway, I ONLY take the PCSK9 inhibitor and my numbers are as follows.... I'm 47 years old with familial hypercholesterolemia. Before the PCSK9 inhibitors: Total: 316, Triglycerides: 252, HDL: 42, LDL: 224; Post the PCSK9 Inhibitor: Total: 159, Triglycerides: 107, HDL: 57, LDL: 81."
    }
    
    Output:
    {
        "entities": [
        {
            "label": "User",
            "properties": {
                "name": "chrisenvy",
                "score": 1,
                "date": "2024-04-04"
            }
        },
        {
            "label": "Condition",
            "properties": {
                "name": "Allergic to statins",
                "description": "User reports being allergic to statins and experiencing liver damage when taking them."
            }
        },
        {
            "label": "Condition",
            "properties": {
                "name": "Familial hypercholesterolemia",
                "description": "User reports having familial hypercholesterolemia."
            }
        },
        {
            "label": "Condition",
            "properties": {
                "name": "Decrease in cholesterol levels after PCSK9 inhibitor",
                "description": "User's total cholesterol decreased from 316 to 159, triglycerides decreased from 252 to 107, HDL increased from 42 to 57, and LDL decreased from 224 to 81 after using PCSK9 inhibitors."
            }
        },
        {
            "label": "Insurance",
            "properties": {
                "name": "Insurance switch",
                "description": "User switched insurance providers because each only covered one of the medications (Praluent or Repatha)."
            }
        },
        {
            "label": "Medication",
            "properties": {
                "name": "Praluent",
                "description": "User currently takes Praluent, a PCSK9 inhibitor."
            }
        },
        {
            "label": "Medication",
            "properties": {
                "name": "Repatha",
                "description": "User previously took Repatha, a PCSK9 inhibitor."
            }
        }
    ],
    "relationships": [
        {
            "start": {
                "label": "User",
                "properties": {"name": "chrisenvy"}
            },
            "end": {
                "label": "Condition",
                "properties": {
                    "name": "Allergic to statins"
                }
            },
            "type": "SUFFERS_FROM"
        },
        {
            "start": {
                "label": "User",
                "properties": {"name": "chrisenvy"}
            },
            "end": {
                "label": "Condition",
                "properties": {
                    "name": "Familial hypercholesterolemia"
                }
            },
            "type": "SUFFERS_FROM"
        },
        {
            "start": {
                "label": "User",
                "properties": {"name": "chrisenvy"}
            },
            "end": {
                "label": "Condition",
                "properties": {
                    "name": "Decrease in cholesterol levels after PCSK9 inhibitor"
                }
            },
            "type": "EXPERIENCED"
        },
        {
            "start": {
                "label": "User",
                "properties": {"name": "chrisenvy"}
            },
            "end": {
                "label": "Insurance",
                "properties": {
                    "name": "Insurance switch"
                }
            },
            "type": "EXPERIENCED"
        },
        {
            "start": {
                "label": "User",
                "properties": {"name": "chrisenvy"}
            },
            "end": {
                "label": "Medication",
                "properties": {
                    "name": "Praluent"
                }
            },
            "type": "USED"
        },
        {
            "start": {
                "label": "User",
                "properties": {"name": "chrisenvy"}
            },
            "end": {
                "label": "Medication",
                "properties": {
                    "name": "Repatha"
                }
            },
            "type": "USED"
        }
    ]
    }
    
    Example 2:
    
    Input:
    {
    "user_id": "john85259",
    "score": 4,
    "date": "2024-02-16",
    "text": "I was taking 40 MG of Crestor (statin), and my ALT and AST liver enzymes went up by a factor 3 and were still increasing when I switched to Praluent. My ALT and AST levels went back to normal levels and my LDL and total cholesterol decreased to very acceptable levels. I have had no negative reaction to Praluent. One plus is that a couple months after I stopped taking Crestor I noticed that my mental processes seemed to be sharper. I've had good results from taking Praluent. My body seems to handle it very well and my mental processes haven't degraded to infant levels yet. Your mileage may vary. Sometimes we have to try one or two different drugs for a particular ailment to find the one that our body can handle. How your body will tolerate a PCSK9 inhibitor is something you won't find out until you start taking it. I think they've had good results overall and I've certainly benefited from taking Praluent but we're all different so your response to it is something you'll want to be sensitive to. Statin drugs are dirt cheap. Praluent isn't cheap and won't come off patent for quite a few years. Nominal price is around $550 a month. Hopefully your insurance will cover all or most of this. It comes in a syringe that you inject every 15 days. It comes in a pen that you press against your skin in a fatty location and push a button. The pen pokes the needle into your skin and injects the fluid. Takes maybe 3-4 seconds. Super easy if you don't have an irrational fear of needles and that sort of thing."
    }
    
    Output:
    "entities": [
        {
            "label": "User",
            "properties": {
                "user_id": "john85259",
                "score": 4,
                "date": "2024-02-16"
            }
        },
        {
            "label": "Condition",
            "properties": {
                "name": "Liver enzyme elevation",
                "description": "ALT and AST liver enzymes increased by a factor of 3 while taking Crestor."
            }
        },
        {
            "label": "Condition",
            "properties": {
                "name": "Normalized liver enzyme levels after Praluent",
                "description": "ALT and AST levels returned to normal after switching to Praluent."
            }
        },
        {
            "label": "Condition",
            "properties": {
                "name": "Improved mental sharpness",
                "description": "Mental processes became sharper after stopping Crestor."
            }
        },
        {
            "label": "Condition",
            "properties": {
                "name": "Decrease in cholesterol levels after Praluent",
                "description": "LDL and total cholesterol decreased to very acceptable levels after using Praluent."
            }
        },
        {
            "label": "Attribute",
            "properties": {
                "name": "Cost of Praluent",
                "description": "Praluent costs $550 per month and is expensive compared to statins."
            }
        },
        {
            "label": "Attribute",
            "properties": {
                "name": "Praluent administration method",
                "description": "Praluent is administered via a syringe or pen every 15 days."
            }
        },
        {
            "label": "Insurance",
            "properties": {
                "name": "Insurance coverage for Praluent",
                "description": "Insurance might cover all or most of the cost of Praluent."
            }
        },
        {
            "label": "Medication",
            "properties": {
                "name": "Praluent",
                "description": "User currently takes Praluent, a PCSK9 inhibitor."
            }
        },
        {
            "label": "Medication",
            "properties": {
                "name": "Crestor",
                "description": "User previously took Crestor, a statin."
            }
        }
    ],
    "relationships": [
        {
            "start": {
                "label": "User",
                "properties": {"name": "john85259"}
            },
            "end": {
                "label": "Condition",
                "properties": {
                    "name": "Liver enzyme elevation"
                }
            },
            "type": "SUFFERS_FROM"
        },
        {
            "start": {
                "label": "User",
                "properties": {"name": "john85259"}
            },
            "end": {
                "label": "Condition",
                "properties": {
                    "name": "Normalized liver enzyme levels after Praluent"
                }
            },
            "type": "EXPERIENCED"
        },
        {
            "start": {
                "label": "User",
                "properties": {"name": "john85259"}
            },
            "end": {
                "label": "Condition",
                "properties": {
                    "name": "Improved mental sharpness"
                }
            },
            "type": "EXPERIENCED"
        },
        {
            "start": {
                "label": "User",
                "properties": {"name": "john85259"}
            },
            "end": {
                "label": "Condition",
                "properties": {
                    "name": "Decrease in cholesterol levels after Praluent"
                }
            },
            "type": "EXPERIENCED"
        },
        {
            "start": {
                "label": "User",
                "properties": {"name": "john85259"}
            },
            "end": {
                "label": "Attribute",
                "properties": {
                    "name": "Cost of Praluent"
                }
            },
            "type": "MENTIONS"
        },
        {
            "start": {
                "label": "User",
                "properties": {"name": "john85259"}
            },
            "end": {
                "label": "Attribute",
                "properties": {
                    "name": "Praluent administration method"
                }
            },
            "type": "MENTIONS"
        },
        {
            "start": {
                "label": "User",
                "properties": {"name": "john85259"}
            },
            "end": {
                "label": "Insurance",
                "properties": {
                    "name": "Insurance coverage for Praluent"
                }
            },
            "type": "MENTIONS"
        },
        {
            "start": {
                "label": "User",
                "properties": {"name": "john85259"}
            },
            "end": {
                "label": "Medication",
                "properties": {
                    "name": "Praluent"
                }
            },
            "type": "USED"
        },
        {
            "start": {
                "label": "User",
                "properties": {"name": "john85259"}
            },
            "end": {
                "label": "Medication",
                "properties": {
                    "name": "Crestor"
                }
            },
            "type": "USED"
        }
    ]
    """
}

In [234]:
cypher = """
MATCH (n)
RETURN n
"""

In [226]:
cypher = """
MATCH (n)
DETACH DELETE n
"""

In [235]:
kg.query(cypher)

[{'n': {'name': 'High cholesterol',
   'description': 'User had a high cholesterol level of 600 before starting Repatha.'}},
 {'n': {'name': 'Decrease in cholesterol levels after Repatha',
   'description': "User's cholesterol decreased from a high of 600 to 120 after using Repatha."}},
 {'n': {'name': 'Side effects of statins',
   'description': 'User experienced side effects from all statins tried before switching to Repatha.'}},
 {'n': {'name': 'Blood pressure and blood sugar increase',
   'description': 'User thinks that Repatha raised their blood pressure and blood sugar a bit.'}},
 {'n': {'date': '2024-01-27', 'score': 1, 'name': 'tinkerbellshefell'}},
 {'n': {'name': 'Medicare',
   'description': 'User started Medicare on January 1, 2024, affecting their ability to use the co-pay card.'}},
 {'n': {'name': 'MediGap',
   'description': 'User has supplemental MediGap insurance through AARP/United Healthcare.'}},
 {'n': {'name': 'Co-pay cost',
   'description': "User's co-pay for me

In [5]:
kg.refresh_schema()
print(kg.schema)

Node properties:
User {name: STRING, date: STRING, score: INTEGER}
Condition {name: STRING, description: STRING, embedding: LIST}
Insurance {name: STRING, description: STRING, embedding: LIST}
Medication {name: STRING, description: STRING, embedding: LIST}
Attribute {name: STRING, description: STRING, embedding: LIST}
Task {name: STRING, description: STRING, embedding: LIST}
Relationship properties:

The relationships:
(:User)-[:USED]->(:Medication)
(:User)-[:USED]->(:Task)
(:User)-[:MENTIONS]->(:Insurance)
(:User)-[:MENTIONS]->(:Task)
(:User)-[:MENTIONS]->(:Attribute)
(:User)-[:SUFFERS_FROM]->(:Condition)
(:User)-[:SUFFERS_FROM]->(:Task)
(:User)-[:EXPERIENCED]->(:Attribute)
(:User)-[:EXPERIENCED]->(:Task)
(:User)-[:EXPERIENCED]->(:Condition)


In [ ]:
def insert_data_to_neo4j(dic):
    with driver.session() as session:
    # Insert nodes
    for entity in dic["entities"]:
        query = f"""
        MERGE (n:{entity['label']} {{name: $name}})
        SET n += $properties
        """
        session.run(query, name=entity["properties"]["name"], properties=entity["properties"])
    
    # Insert relationships
    for relationship in dic["relationships"]:
        start_label = relationship["start"]["label"]
        start_props = relationship["start"]["properties"]
        end_label = relationship["end"]["label"]
        end_props = relationship["end"]["properties"]

        query = f"""
        MATCH (start:{start_label} {{name: $start_name}})
        MATCH (end:{end_label} {{name: $end_name}})
        MERGE (start)-[r:{relationship['type']}]->(end)
        """
        session.run(
            query,
            start_name=start_props["name"],
            end_name=end_props["name"]
        )

In [171]:
text1 = """
    {
    "id": "jahozer1",
    "score": 6,
    "date": "2022-12-09"
    "text": "I have been on it for about a year. Very happy with it. I tried all the statins and had the side effects. I dont feel any with Repatha, and my cholesterol is down from a high of 600 to 120. I do think it raised my blood pressure and blood sugar a bit."
    }
"""

In [172]:
message = prompt["Task"].format(posting=text1) + prompt["Instruction"]
dic1 = json.loads(call_gpt4o_mini(message))

In [173]:
print(dic1)

{'entities': [{'label': 'User', 'properties': {'name': 'jahozer1', 'score': 6, 'date': '2022-12-09'}}, {'label': 'Medication', 'properties': {'name': 'Repatha', 'description': 'User has been taking Repatha, a PCSK9 inhibitor, for about a year.'}}, {'label': 'Condition', 'properties': {'name': 'High cholesterol', 'description': 'User had a high cholesterol level of 600 before starting Repatha.'}}, {'label': 'Condition', 'properties': {'name': 'Decrease in cholesterol levels after Repatha', 'description': "User's cholesterol decreased from a high of 600 to 120 after using Repatha."}}, {'label': 'Attribute', 'properties': {'name': 'Side effects of statins', 'description': 'User experienced side effects from all statins tried before switching to Repatha.'}}, {'label': 'Attribute', 'properties': {'name': 'Blood pressure and blood sugar increase', 'description': 'User thinks that Repatha raised their blood pressure and blood sugar a bit.'}}], 'relationships': [{'start': {'label': 'User', 'pr

In [229]:
insert_data_to_neo4j(dic1)

In [178]:
text2 = """
    {
    "id": "tinkerbellshefell",
    "score": 1,
    "date": "2024-01-27"
    "text": "I used the Repatha co-pay card last year and only paid $5 for 2 pens each month. I started Medicare on January 1 this year and can no longer get the card. I have supplemental MediGap through AARP/United Healthcare. My pharmacy just told me my co-pay is $553.98! There's just no way I can get my meds without a co-pay card."
    }
"""

In [179]:
message = prompt["Task"].format(posting=text2) + prompt["Instruction"]
dic2 = json.loads(call_gpt4o_mini(message))

In [180]:
print(dic2)

{'entities': [{'label': 'User', 'properties': {'name': 'tinkerbellshefell', 'score': 1, 'date': '2024-01-27'}}, {'label': 'Medication', 'properties': {'name': 'Repatha', 'description': 'User used Repatha co-pay card to reduce medication costs.'}}, {'label': 'Insurance', 'properties': {'name': 'Medicare', 'description': 'User started Medicare on January 1, 2024, affecting their ability to use the co-pay card.'}}, {'label': 'Insurance', 'properties': {'name': 'MediGap', 'description': 'User has supplemental MediGap insurance through AARP/United Healthcare.'}}, {'label': 'Attribute', 'properties': {'name': 'Co-pay cost', 'description': "User's co-pay for medication increased to $553.98 after starting Medicare."}}, {'label': 'Attribute', 'properties': {'name': 'Co-pay card dependency', 'description': 'User expresses inability to afford medication without a co-pay card.'}}], 'relationships': [{'start': {'label': 'User', 'properties': {'name': 'tinkerbellshefell'}}, 'end': {'label': 'Medicat

In [230]:
insert_data_to_neo4j(dic2)

In [184]:
text3 = """
    {
    "id": "rasmith337",
    "score": 4,
    "date": "2024-03-15"
    "text": "I'm right there with you. I took the drug for 18 months. I was getting full body cramps and even into my hands multiple times per day. It was scary. Cardiologist said it was likely electrolytes but it continued. There was a decent amount of pain in my legs and hips too. I was also having bad effects to my memory and mental clarity. It was like I had aged decades in a matter of months. My movement was very slow and I knew I wasn't totally there. Within a couple of weeks of stopping the drug things started improving but I still don't feel like I'm 100% back, and it's been a year now. My wife did say it was like she could see the fog lifted from over me. She didn't say it while I was taking it but she was getting scared. My cardiologist wanted me to continue but I decided to stop, and find a new cardiologist."
    }
"""

In [185]:
message = prompt["Task"].format(posting=text3) + prompt["Instruction"]
dic3 = json.loads(call_gpt4o_mini(message))

In [186]:
print(dic3)

{'entities': [{'label': 'User', 'properties': {'name': 'rasmith337', 'score': 4, 'date': '2024-03-15'}}, {'label': 'Condition', 'properties': {'name': 'Full body cramps', 'description': 'User experienced full body cramps multiple times per day while taking the medication.'}}, {'label': 'Condition', 'properties': {'name': 'Pain in legs and hips', 'description': 'User reported experiencing pain in legs and hips while on the medication.'}}, {'label': 'Condition', 'properties': {'name': 'Memory and mental clarity issues', 'description': 'User experienced bad effects on memory and mental clarity, feeling like they aged decades.'}}, {'label': 'Condition', 'properties': {'name': 'Slow movement', 'description': 'User noted that their movement was very slow while taking the medication.'}}, {'label': 'Attribute', 'properties': {'name': 'Improvement after stopping medication', 'description': 'User reported improvement in symptoms within a couple of weeks after stopping the medication.'}}, {'label

In [231]:
insert_data_to_neo4j(dic3)

In [190]:
text4 = """
    {
    "id": "solidrock80",
    "score": 4,
    "date": "2024-02-04"
    "text": "I was on Praluent for a year and a half, now Repatha after getting switched by insurer. 59 M and 165 lbs. I had a lot of muscle pain and discomfort on three statins before I tried Praluent. I was scared of side effects and almost didn't do it. So glad I did - no side effects at all, and reduced my ldl from 160 to the 50s. It's easy to fall into the trap of fear about side effects. But doing nothing given your very high LDL is a choice with a whole set of medical effects. FWIW from what I see on the FB Repatha group, it seems like people who inject in leg or arm complain of more symptoms. I inject in my belly."
    }
"""

In [191]:
message = prompt["Task"].format(posting=text4) + prompt["Instruction"]
dic4 = json.loads(call_gpt4o_mini(message))

In [192]:
print(dic4)

{'entities': [{'label': 'User', 'properties': {'name': 'solidrock80', 'score': 4, 'date': '2024-02-04'}}, {'label': 'Medication', 'properties': {'name': 'Praluent', 'description': 'User took Praluent for a year and a half.'}}, {'label': 'Medication', 'properties': {'name': 'Repatha', 'description': 'User switched to Repatha after being switched by insurer.'}}, {'label': 'Condition', 'properties': {'name': 'Muscle pain and discomfort from statins', 'description': 'User experienced muscle pain and discomfort on three statins before trying Praluent.'}}, {'label': 'Condition', 'properties': {'name': 'High LDL', 'description': 'User had a very high LDL before starting treatment.'}}, {'label': 'Attribute', 'properties': {'name': 'Fear of side effects', 'description': 'User was scared of side effects before trying Praluent.'}}, {'label': 'Attribute', 'properties': {'name': 'Injection site', 'description': 'User injects Repatha in the belly, noting that others report more symptoms when injecti

In [232]:
insert_data_to_neo4j(dic4)

In [194]:
text5 = """
    {
    "id": "MathSlow946",
    "score": 4,
    "date": "2022-12-10"
    "text": "F Repatha! I took three shots over 6 weeks and every time within a few days fatigue, stomach pain, no appetite, crushing back and joint pain, weakness! At the end of the 2 week intervals I felt better, but the next dose would send me back to a complete state of ill health. I went from feeling crippled to back in the gym for the first time since I started this med. Back on Crestor EOD to help the muscle pain and I’m ok so far! Repatha was touted as being the answer to statin side effects. Not for me and many others. Worse"
    }
"""

In [195]:
message = prompt["Task"].format(posting=text5) + prompt["Instruction"]
dic5 = json.loads(call_gpt4o_mini(message))

In [196]:
print(dic5)

{'entities': [{'label': 'User', 'properties': {'name': 'MathSlow946', 'score': 4, 'date': '2022-12-10'}}, {'label': 'Medication', 'properties': {'name': 'Repatha', 'description': 'User took Repatha, a PCSK9 inhibitor, but experienced negative side effects.'}}, {'label': 'Medication', 'properties': {'name': 'Crestor', 'description': 'User is currently taking Crestor to help with muscle pain.'}}, {'label': 'Condition', 'properties': {'name': 'Fatigue', 'description': 'User experienced fatigue after taking Repatha.'}}, {'label': 'Condition', 'properties': {'name': 'Stomach pain', 'description': 'User experienced stomach pain after taking Repatha.'}}, {'label': 'Condition', 'properties': {'name': 'Loss of appetite', 'description': 'User experienced a loss of appetite after taking Repatha.'}}, {'label': 'Condition', 'properties': {'name': 'Joint pain', 'description': 'User experienced crushing back and joint pain after taking Repatha.'}}, {'label': 'Condition', 'properties': {'name': 'Weakn

In [233]:
insert_data_to_neo4j(dic5)